In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import sys
# 绘图
%matplotlib inline
from matplotlib import pyplot as plt
tf.enable_eager_execution()

In [ ]:
class PNet(tf.keras.Model):
    def __init__(self):
        super(PNet, self).__init__()
        self.conv1 = conv2d(10, 3, 1, 'valid', name='conv1')
        self.prelu1 = tf.keras.layers.PReLU(name='prelu1',shared_axes=[1,2])
        self.pool1 = pool2d(name='pool1')

        self.conv2 = conv2d(16, 3, 1, 'valid', name='conv2')
        self.prelu2 = tf.keras.layers.PReLU(name='prelu2',shared_axes=[1,2])

        self.conv3 = conv2d(32, 3, 1, 'valid', name='conv3')
        self.prelu3 = tf.keras.layers.PReLU(name='prelu3',shared_axes=[1,2])

        self.conv4_1 = conv2d(
            2, 1, 1, 'same', activation='softmax', name='conv4_1')
        self.conv4_2 = conv2d(4, 1, 1, 'same', name='conv4_2')

    def call(self, input_):
        out = self.conv1(input_)
        out = self.prelu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.prelu2(out)
        out = self.conv3(out)
        out = self.prelu3(out)

        prob = self.conv4_1(out)
        loc = self.conv4_2(out)
        return prob, loc
    def model_variable_initialize(self):
        image = tf.random_normal((1, 12, 12, 3))
        with tf.name_scope('PNet'):
            self.call(image)
        print("Completed")

def conv2d(filter, ksize=3, stride=1, padding='same', dilation=1, activation=None, name="conv2d"):
    ksize = [ksize] * 2
    strides = [stride] * 2
    dilation = [dilation] * 2
    return tf.keras.layers.Conv2D(filters=filter, kernel_size=ksize, strides=strides, padding=padding, dilation_rate=dilation, activation=activation, name=name)


def pool2d(ksize=2, stride=2, padding='same', name='pool2d'):
    ksize = [ksize] * 2
    strides = [stride]*2
    return tf.keras.layers.MaxPool2D(pool_size=ksize, strides=strides, padding=padding, name=name)


In [ ]:
model = PNet()
model.model_variable_initialize()
model_vars = model.variables

In [ ]:
image = cv2.imread('other_02.jpg')
"""
image = (image-127.5)/128
image = np.expand_dims(image,0)
model = PNet()
model(image)
"""

In [ ]:
weights = np.load(r"F:\Script\models\face recognition\det1.npy",encoding='latin1').item()
to = {'prelu1': 'PReLU1', 'prelu2': 'PReLU2', 'prelu3': 'PReLU3', 'conv1': 'conv1',
      'conv2': 'conv2', 'conv3': 'conv3', 'conv4_1': 'conv4-1', 'conv4_2': 'conv4-2'}
too = {'kernel':'weights','bias':'biases','alpha':'alpha'}
for var in model_vars:
    name = var.name
    name0 = name.split('/')[1]
    name1 = name.split('/')[-1]
    name1 = name1.split(':')[0]
    dual_name0 = to[name0]
    dual_name1 = too[name1]
    value = weights[dual_name0][dual_name1]
    if name1 == "alpha":
        value = np.expand_dims(value,0)
        value = np.expand_dims(value,0)
    print(name0,name1,var.shape,value.shape)
    tf.assign(var,value)

In [ ]:
model.save_weights("./checkpoints/pnet/model")

In [ ]:
prob,loc = model(image)

In [ ]:
tf.reduce_sum(tf.argmax(prob,axis=-1))

In [ ]:
images, scales = generate_samples_from_image(image, 30)
img = images[0]
scale = scales[0]

In [ ]:
total_bboxes = []
total_bboxes_ref = []
for img, scale in zip(images, scales):
    prob, loc = model(img)
    original_bboxes = generate_original_boxes(loc.numpy(), scale)
    filter_mask = tf.argmax(prob, axis=-1)
    bboxes_tf = tf.boolean_mask(original_bboxes, filter_mask)
    bboxes_ref_tf = tf.boolean_mask(loc, filter_mask)
    local_bboxes, local_bboxes_ref = bboxes_nms(
        bboxes_tf.numpy(), bboxes_ref_tf.numpy(), 0.6)
    total_bboxes.append(local_bboxes)
    total_bboxes_ref.append(local_bboxes_ref)

In [ ]:
total_bboxes = np.concatenate(total_bboxes)
total_bboxes_ref = np.concatenate(total_bboxes_ref)
bboxes, bboxes_ref = bboxes_nms(total_bboxes, total_bboxes_ref, 0.7)
bboxes = bboxes_reg(bboxes, bboxes_ref)

In [ ]:
h, w, _ = image.shape
bboxes_cp, pad_bboxes = bboxes_clip(bboxes, w, h)

In [ ]:
ii = crop_image(image,bboxes_cp, pad_bboxes)

In [ ]:
ii.shape

In [ ]:
i = 0
keep_bboxes = np.ones((bboxes.shape[0], 1), dtype=np.bool)
overlap = bboxes_iou(bboxes[i], bboxes[(i+1):])

In [ ]:
keep_overlap = overlap < 0.6

In [ ]:
keep_bboxes[(i+1):] = np.logical_and(keep_bboxes[(i+1):],np.expand_dims(keep_overlap,1))

In [ ]:
keep_overlap.shape

In [ ]:
class RNet(tf.keras.Model):
    def __init__(self):
        super(RNet, self).__init__()
        self.conv1 = conv2d(28, 3, 1, 'valid', name='conv1')
        self.prelu1 = tf.keras.layers.PReLU(name='prelu1', shared_axes=[1, 2])
        self.pool1 = pool2d(3, 2, name='pool1')

        self.conv2 = conv2d(48, 3, 1, 'valid', name='conv2')
        self.prelu2 = tf.keras.layers.PReLU(name='prelu2', shared_axes=[1, 2])
        self.pool2 = pool2d(3, 2, 'valid', name='pool2')

        self.conv3 = conv2d(64, 2, 1, 'valid', name='conv3')
        self.prelu3 = tf.keras.layers.PReLU(name='prelu3', shared_axes=[1, 2])

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, name='fc1')
        self.prelu4 = tf.keras.layers.PReLU(name='prelu4')

        self.fc2_1 = tf.keras.layers.Dense(
            2, activation='softmax', name='fc2_1')
        self.fc2_2 = tf.keras.layers.Dense(4, name='fc2_2')

    def call(self, input_):
        out = self.conv1(input_)
        out = self.prelu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.prelu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.prelu3(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.prelu4(out)

        prob = self.fc2_1(out)
        loc = self.fc2_2(out)
        return prob, loc

    def model_variable_initialize(self):
        image = tf.random_normal((1, 24, 24, 3))
        with tf.name_scope('RNet'):
            self.call(image)
        print("Completed")


In [ ]:
model = RNet()

In [ ]:
model.model_variable_initialize()

In [ ]:
model_vars = model.variables

In [ ]:
for var in model_vars:
    print(var.name,var.shape)

In [ ]:
weights = np.load(r"F:\Script\models\face recognition\det2.npy",encoding='latin1').item()

In [ ]:
sorted(weights.keys())

In [ ]:
to = {'prelu1': 'prelu1', 'prelu2': 'prelu2', 'prelu3': 'prelu3','prelu4': 'prelu4', 'conv1': 'conv1',
      'conv2': 'conv2', 'conv3': 'conv3', 'fc1': 'conv4', 'fc2_1': 'conv5-1', 'fc2_2': 'conv5-2'}
too = {'kernel':'weights','bias':'biases','alpha':'alpha'}
for var in model_vars:
    name = var.name
    name0 = name.split('/')[1]
    name1 = name.split('/')[-1]
    name1 = name1.split(':')[0]
    dual_name0 = to[name0]
    dual_name1 = too[name1]
    value = weights[dual_name0][dual_name1]
    if name1 == "alpha" and name0 != 'prelu4':
        value = np.expand_dims(value,0)
        value = np.expand_dims(value,0)
    print(name0,name1,var.shape,value.shape)
    tf.assign(var,value)

In [ ]:
model.save_weights("./checkpoints/rnet/model")

In [ ]:
class ONet(tf.keras.Model):
    def __init__(self):
        super(ONet, self).__init__()
        self.conv1 = conv2d(32, 3, 1, 'valid', name='conv1')
        self.prelu1 = tf.keras.layers.PReLU(name='prelu1', shared_axes=[1, 2])
        self.pool1 = pool2d(3, 2, name='pool1')

        self.conv2 = conv2d(64, 3, 1, 'valid', name='conv2')
        self.prelu2 = tf.keras.layers.PReLU(name='prelu2', shared_axes=[1, 2])
        self.pool2 = pool2d(3, 2, 'valid', name='pool2')

        self.conv3 = conv2d(64, 3, 1, 'valid', name='conv3')
        self.prelu3 = tf.keras.layers.PReLU(name='prelu3', shared_axes=[1, 2])
        self.pool3 = pool2d(2, 2, name='pool3')

        self.conv4 = conv2d(128, 2, 1, 'valid', name='conv4')
        self.prelu4 = tf.keras.layers.PReLU(name='prelu4', shared_axes=[1, 2])

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, name='fc1')
        self.prelu5 = tf.keras.layers.PReLU(name='prelu5')

        self.fc2_1 = tf.keras.layers.Dense(
            2, activation='softmax', name='fc2_1')
        self.fc2_2 = tf.keras.layers.Dense(4, name='fc2_2')
        self.fc2_3 = tf.keras.layers.Dense(10, name='fc2_3')

    def call(self, input_):
        out = self.conv1(input_)
        out = self.prelu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.prelu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.prelu3(out)
        out = self.pool3(out)
        out = self.conv4(out)
        out = self.prelu4(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.prelu5(out)

        prob = self.fc2_1(out)
        loc = self.fc2_2(out)
        landmark = self.fc2_3(out)
        return prob, loc, landmark

    def model_variable_initialize(self):
        image = tf.random_normal((1, 48, 48, 3))
        with tf.name_scope('ONet'):
            self.call(image)
        print("Completed")


In [ ]:
model = ONet()
model.model_variable_initialize()
model_vars = model.variables

In [ ]:
for var in model_vars:
    print(var.name,var.shape)

In [ ]:
weights = np.load(r"F:\Script\models\face recognition\det3.npy",encoding='latin1').item()
sorted(weights.keys())

In [ ]:
to = {'prelu1': 'prelu1', 'prelu2': 'prelu2', 'prelu3': 'prelu3','prelu4': 'prelu4', 'prelu5': 'prelu5', 'conv1': 'conv1',
      'conv2': 'conv2', 'conv3': 'conv3', 'conv4': 'conv4', 'fc1': 'conv5', 'fc2_1': 'conv6-1', 'fc2_2': 'conv6-2', 'fc2_3': 'conv6-3'}
too = {'kernel':'weights','bias':'biases','alpha':'alpha'}
for var in model_vars:
    name = var.name
    name0 = name.split('/')[1]
    name1 = name.split('/')[-1]
    name1 = name1.split(':')[0]
    dual_name0 = to[name0]
    dual_name1 = too[name1]
    value = weights[dual_name0][dual_name1]
    if name1 == "alpha" and name0 != 'prelu5':
        value = np.expand_dims(value,0)
        value = np.expand_dims(value,0)
    print(name0,name1,var.shape,value.shape)
    tf.assign(var,value)

In [ ]:
model.save_weights("./checkpoints/onet/model")

In [ ]:
def resize_image(image, size=(12, 12)):
    return cv2.resize(image, size)


def preprocess_image(image):
    """Preprocess a image"""
    image_cp = np.copy(image)#.astype(np.float32)
    # regularize the image
    image_regular = (image_cp-127.5)/128
    # expand the batch_size dim
    image_expanded = np.expand_dims(image_regular, axis=0)
    return image_expanded
def cal_scales(minsize, factor=0.709, size=(12, 12)):
    minlen = np.min(size)
    s = 12.0/minsize
    minlen *= (s*factor)
    scales = [s]
    while (minlen >= 12):
        scales.append(s * np.power(factor, len(scales)))
        minlen *= factor
    return scales


def generate_samples_from_image(image, minsize=30, factor=0.709):
    height, weight, _ = image.shape
    scales = cal_scales(minsize, factor, size=(weight, height))
    images = []
    for scale in scales:
        w, h = int(np.ceil(weight * scale)), int(np.ceil(height * scale))
        images.append(preprocess_image(resize_image(image, size=(h, w))))
    return images, scales


def generate_original_boxes(bbox_reg, scale):
    # bboxes shape of (h,w,4)
    bboxes = np.zeros_like(bbox_reg, dtype=np.float32)
    _,h, w, _ = np.shape(bboxes)
    x, y = np.mgrid[0:w, 0:h]
    bboxes[0, :, :, 0] = x * 2 / scale
    bboxes[0, :, :, 1] = y * 2 / scale
    bboxes[0, :, :, 2] = (x * 2+12) / scale
    bboxes[0, :, :, 3] = (y * 2+12) / scale
    return tf.convert_to_tensor(bboxes)


def bboxes_iou(bboxes1, bboxes2):
    """Computing iou between bboxes1 and bboxes2.
    Note: bboxes1 and bboxes2 can be multi-dimensional, but should broacastable.
    """
    bboxes1 = np.transpose(bboxes1)
    bboxes2 = np.transpose(bboxes2)
    # Intersection bbox and volume.
    int_ymin = np.maximum(bboxes1[0], bboxes2[0])
    int_xmin = np.maximum(bboxes1[1], bboxes2[1])
    int_ymax = np.minimum(bboxes1[2], bboxes2[2])
    int_xmax = np.minimum(bboxes1[3], bboxes2[3])

    int_h = np.maximum(int_ymax - int_ymin, 0.)
    int_w = np.maximum(int_xmax - int_xmin, 0.)
    int_vol = int_h * int_w
    # Union volume.
    vol1 = (bboxes1[2] - bboxes1[0]) * (bboxes1[3] - bboxes1[1])
    vol2 = (bboxes2[2] - bboxes2[0]) * (bboxes2[3] - bboxes2[1])
    iou = int_vol / (vol1 + vol2 - int_vol)
    return iou


def bboxes_nms(bboxes, bboxes_ref, nms_threshold=0.5):
    """Apply non-maximum selection to bounding boxes.
    """
    keep_bboxes = np.ones((bboxes.shape[0]), dtype=np.bool)
    for i in range(bboxes.shape[0]-1):
        if keep_bboxes[i]:
            # Computer overlap with bboxes which are following.
            overlap = bboxes_iou(bboxes[i], bboxes[(i+1):])
            # Overlap threshold for keeping + checking part of the same class
            keep_overlap = overlap < nms_threshold
            keep_bboxes[(i+1):] = np.logical_and(keep_bboxes[(i+1):],
                                                 keep_overlap)
    idxes = np.where(keep_bboxes)
    return bboxes[idxes], bboxes_ref[idxes]


def bboxes_reg(bboxes, bboxes_ref):
    """boxes bounding regression.

    Args:
        bboxes ([array]): original boxes
        bboxes_ref ([array]): [description]

    Returns:
        [array]: [description]
    """
    w = bboxes[:, 0] - bboxes[:, 2]
    h = bboxes[:, 1] - bboxes[:, 3]
    bboxes[:, 0] += bboxes_ref[:, 0] * w
    bboxes[:, 1] += bboxes_ref[:, 1] * h
    bboxes[:, 2] += bboxes_ref[:, 2] * w
    bboxes[:, 3] += bboxes_ref[:, 3] * h
    return bboxes


def rec2square(bboxes):
    """Convert bbox to square."""
    h = bboxes[:, 3]-bboxes[:, 1]
    w = bboxes[:, 2]-bboxes[:, 0]
    l = np.maximum(w, h)
    bboxes[:, 0] = bboxes[:, 0]+w*0.5-l*0.5
    bboxes[:, 1] = bboxes[:, 1]+h*0.5-l*0.5
    bboxes[:, 2:4] = bboxes[:, 0:2] + np.transpose(np.tile(l, (2, 1)))
    return bboxes


def bboxes_clip(bboxes, w, h):
    """Clip bounding boxes

    Args:
        bboxes ([array]): shape of (N, 4)
        w ([int]): the width of input image
        h ([int]): the heigh of input image

    Returns:
        [tuple of array]: return the position of cliped boxes and
        padding for each axis
    """
    bboxes_cp = bboxes.copy()
    boxes_w = (bboxes[:, 2]-bboxes[:, 0]+1).astype(np.int32)
    boxes_h = (bboxes[:, 3] - bboxes[:, 0] + 1).astype(np.int32)
    bboxes_cp[:, 0] = np.maximum(bboxes[:, 0], 0)
    bboxes_cp[:, 1] = np.maximum(bboxes[:, 1], 0)
    bboxes_cp[:, 2] = np.minimum(bboxes[:, 2], w)
    bboxes_cp[:, 3] = np.minimum(bboxes[:, 3], h)

    pad_bboxes = np.zeros_like(bboxes, dtype=np.int32)
    pad_bboxes[:, 0] = bboxes_cp[:, 0] - bboxes[:, 0]
    pad_bboxes[:, 1] = bboxes_cp[:, 1] - bboxes[:, 1]
    pad_bboxes[:, 2] = bboxes[:, 2] - bboxes_cp[:, 2]
    pad_bboxes[:, 3] = bboxes[:, 3] - bboxes_cp[:, 3]
    pad_bboxes[pad_bboxes < 0] = 0
    return bboxes_cp.astype(np.int32), pad_bboxes


def crop_image(image, bboxes, pad_bboxes, size=[24, 24]):
    # example shape of [N,24,24,3]
    shape = [bboxes.shape[0]] + size + [3]
    images = np.zeros(shape)
    for idx in range(bboxes.shape[0]):
        x1, y1, x2, y2 = bboxes[idx]
        padding = ((pad_bboxes[idx, 1], pad_bboxes[idx, 3]),
                   (pad_bboxes[idx, 0], pad_bboxes[idx, 2]),
                   (0,0))
        temp_img = image[x1:x2, y1:y2]
        temp_img = np.pad(
            temp_img, padding, 'constant', constant_values=0)
        images[idx] = resize_image(temp_img, size=tuple(size))
    return images
